In [1]:
from elexon import ElexonRawClient
from calendar import monthrange
import datetime
import pandas as pd
from tabulate import tabulate
from tqdm import tqdm
from sys import exit
from tqdm import tqdm
import os 

In [2]:
MarketIndexData = pd.read_excel("0.DataSource/PricesAndVolume.xlsx",sheet_name='MarketIndexData')
SBuyData = pd.read_excel("0.DataSource/PricesAndVolume.xlsx",sheet_name='SystemBuySellData')
MarketIndexData['Settlement Date'] = pd.to_datetime(MarketIndexData['Settlement Date'])
SBuyData['Settlement Date'] = pd.to_datetime(SBuyData['Settlement Date'])

In [3]:
MarketIndexData

,Settlement Date,Settlement Period,Market Index Data Provider Id,Market Index Volume (MWh),Market Index Price (GBP/MWh)
0,2014-01-01,1,APXMIDP,301.50,35.32
1,2014-01-01,1,N2EXMIDP,0.00,0.00
2,2014-01-01,2,APXMIDP,365.65,37.28
3,2014-01-01,2,N2EXMIDP,0.00,0.00
4,2014-01-01,3,APXMIDP,184.00,41.50
...,...,...,...,...,...
358219,2024-03-21,46,N2EXMIDP,0.00,0.00
358220,2024-03-21,47,APXMIDP,1653.20,25.19
358221,2024-03-21,47,N2EXMIDP,0.00,0.00
358222,2024-03-21,48,APXMIDP,1492.85,17.41


In [4]:
SBuyData

,Settlement Date,Settlement Period,System Sell Price(GBP/MWh),Net Imbalance Volume(MWh)
0,2014-12-16,29,40.09,-357.2920
1,2014-12-16,30,39.10,-268.6010
2,2014-12-16,31,45.32,-247.8930
3,2014-12-16,32,51.00,118.3120
4,2014-12-16,33,54.11,11.0381
...,...,...,...,...
162567,2024-03-25,21,51.14,-116.8500
162568,2024-03-25,22,51.16,-76.1931
162569,2024-03-25,23,51.16,-325.4562
162570,2024-03-25,24,51.16,-301.3287


In [5]:
# Year,Month,Day = 2023,9,17
# MID_Year = MarketIndexData.loc[(MarketIndexData["Settlement Date"].dt.year == Year)] # Market Index Data
# MID_Month = MID_Year.loc[(MID_Year["Settlement Date"].dt.month == Month)]
# MID_Day = MID_Month.loc[(MID_Month["Settlement Date"].dt.day == Day)]
# MID_SP = MID_Day.loc[(MID_Day["Settlement Period"] == 2)]
# #
# MID_Day.shape[0]

In [6]:
# MID_APX = MID_SP.loc[(MID_SP["Market Index Data Provider Id"] == "APXMIDP ")]
# MID_N2EX = MID_SP.loc[(MID_SP["Market Index Data Provider Id"] == "N2EXMIDP")]
# MID_N2EX.shape[0]

In [7]:
# MID_APX_Vol = MID_APX['Market Index Volume (MWh)'].values[0]
# MID_APX_Price = MID_APX['Market Index Price (GBP/MWh)'].values[0]
# MID_N2EX_Vol = MID_N2EX['Market Index Volume (MWh)'].values[0]
# MID_N2EX_Price = MID_N2EX['Market Index Price (GBP/MWh)'].values[0]
# print(MID_APX_Vol,MID_APX_Price,MID_N2EX_Vol,MID_N2EX_Price)

In [8]:
# SBD_Year = SBuyData.loc[(SBuyData["Settlement Date"].dt.year == Year)] # System Buy Data
# SBD_Month = SBD_Year.loc[(SBD_Year["Settlement Date"].dt.month == Month)]
# SBD_Day = SBD_Month.loc[(SBD_Month["Settlement Date"].dt.day == Day)]
# #
# SBD_SP = SBD_Day.loc[(SBD_Day["Settlement Period"] == 47)]
# SBD_SP.shape[0]
#SBD_Price = SBD_SP['System Sell Price(GBP/MWh)'].values[0]
#SBD_Volume = SBD_SP['Net Imbalance Volume(MWh)'].values[0]
#print(SBD_Price,SBD_Volume)

In [9]:
Years = tqdm(range(2015,2023+1))
#Years = range(2015,2023+1)
DataFrame = pd.DataFrame(columns=["Date","Year","Month","Day","SettlementPeriod",
                                  "MID_APX_VOLUME(MWh)","MID_APX_PRICE(GBP/MWh)","MID_N2EX_VOLUME(MWh)","MID_N2EX_PRICE(GBP/MWh)","SSP(GBP/MWh)","NIV(MWh)"])
for Year in Years:
    MID_Year = MarketIndexData.loc[(MarketIndexData["Settlement Date"].dt.year == Year)] # Market Index Data
    SBD_Year = SBuyData.loc[(SBuyData["Settlement Date"].dt.year == Year)]               # System Buy Data
    for Month in range(1,12+1):
        MID_Month = MID_Year.loc[(MID_Year["Settlement Date"].dt.month == Month)]
        SBD_Month = SBD_Year.loc[(SBD_Year["Settlement Date"].dt.month == Month)]
        Days = monthrange(Year,Month)[1]
        for Day in range(1,Days+1):
            Years.set_description(f'{Year}-{Month}-{Day} ##>>')
            Date = datetime.date(Year,Month,Day)
            #DatePrint = f"{Year}-{Month}-{Day}"
            MID_Day = MID_Month.loc[(MID_Month["Settlement Date"].dt.day == Day)]
            SBD_Day = SBD_Month.loc[(SBD_Month["Settlement Date"].dt.day == Day)]
            SettlementPeriods = sorted(MID_Day['Settlement Period'].unique().tolist())
            for Label in SettlementPeriods:
                #Years.set_description(f'{Year}-{Month}-{Day} ## {Label} ##>>')
                DataList = []
                # Market Index Data
                MID_SP = MID_Day.loc[(MID_Day["Settlement Period"] == Label)]
                MID_APX = MID_SP.loc[(MID_SP["Market Index Data Provider Id"] == "APXMIDP ")]
                MID_N2EX = MID_SP.loc[(MID_SP["Market Index Data Provider Id"] == "N2EXMIDP")]
                if MID_APX.shape[0] != 0:
                    MID_APX_Vol = MID_APX['Market Index Volume (MWh)'].values[0]
                    MID_APX_Price = MID_APX['Market Index Price (GBP/MWh)'].values[0]
                else:
                    #print(f"{Year}-{Month}-{Day} ## {Label} ## APXMIDP didn't have any values - Fix IT and rerun the code !!!>>")
                    MID_APX_Vol = 0
                    MID_APX_Price = 0
                #
                if MID_N2EX.shape[0] != 0:
                    MID_N2EX_Vol = MID_N2EX['Market Index Volume (MWh)'].values[0]
                    MID_N2EX_Price = MID_N2EX['Market Index Price (GBP/MWh)'].values[0]
                else:
                    #print(f"{Year}-{Month}-{Day} ## {Label} ## N2EXMIDP didn't have any values - Fix IT and rerun the code !!!>>")
                    MID_N2EX_Vol = 0
                    MID_N2EX_Price = 0
                #
                SBD_SP = SBD_Day.loc[(SBD_Day["Settlement Period"] == Label)]
                if SBD_SP.shape[0] != 0:
                    SBD_Price = SBD_SP['System Sell Price(GBP/MWh)'].values[0]
                    SBD_Volume = SBD_SP['Net Imbalance Volume(MWh)'].values[0]
                else:
                    print(f'{Year}-{Month}-{Day} ## {Label} ## System Buy Price Gap - Fix IT and rerun the code !!!>>')
                    SBD_Price = 0
                    SBD_Volume = 0
                #
                DataList.extend([Date,Year,Month,Day,Label,MID_APX_Vol,MID_APX_Price,MID_N2EX_Vol,MID_N2EX_Price,SBD_Price,SBD_Volume])
                DataFrame.loc[len(DataFrame), DataFrame.columns] = (DataList)
DataFrame.to_csv("0.DataSource/MarketIndexAndSystemBuyData.csv",index=False)
DataFrame

2023-12-31 ##>>: 100%|██████████| 9/9 [35:17<00:00, 235.32s/it]


,Date,Year,Month,Day,SettlementPeriod,MID_APX_VOLUME(MWh),MID_APX_PRICE(GBP/MWh),MID_N2EX_VOLUME(MWh),MID_N2EX_PRICE(GBP/MWh),SSP(GBP/MWh),NIV(MWh)
0,2015-01-01,2015,1,1,1,569.95,37.31,0.0,0.0,37.31,-320.506
1,2015-01-01,2015,1,1,2,511.25,38.2,0.0,0.0,38.2,-168.958
2,2015-01-01,2015,1,1,3,346.45,34.42,0.0,0.0,34.42,-23.876
3,2015-01-01,2015,1,1,4,376.95,30.78,0.0,0.0,30.9611,-419.545
4,2015-01-01,2015,1,1,5,268.65,31.0,0.0,0.0,31.0,-551.878
...,...,...,...,...,...,...,...,...,...,...,...
157771,2023-12-31,2023,12,31,44,1172.25,47.94,0.0,0.0,47.94,17.554
157772,2023-12-31,2023,12,31,45,1155.85,58.39,0.0,0.0,58.39,137.501
157773,2023-12-31,2023,12,31,46,1167.05,44.07,0.0,0.0,8.72,-280.272
157774,2023-12-31,2023,12,31,47,820.35,43.04,0.0,0.0,5.0,-75.634
